In [2]:
import numpy as np
import pandas as pd
import pickle

In [33]:
def remove_NaN(d) :  
    """ Remove rows of a DataFrame with missing firing rate """
    M = np.stack(d['Spike rate'])
    NaN_rows = np.isnan(M).any(axis=1)
    return d.iloc[~NaN_rows,:]


def produce_trial_index(d) :
    """Produce a dictionnary of trial indices corresponding to each label."""
    
    trial_idx = None
    
    go_hit_idx = (d['Choice'] == 1)&(d['Category ID'] == 1)&(d['Context'] == 0)
    go_error_idx = (d['Choice'] == 1)&(d['Category ID'] == 0)&(d['Context'] == 0)

    nogo_hit_idx = (d['Choice'] == 0)&(d['Category ID'] == 0)&(d['Context'] == 0)
    nogo_error_idx = (d['Choice'] == 0)&(d['Category ID'] == 1)&(d['Context'] == 0)

    go2_hit_idx = (d['Choice'] == 1) & (d['Category ID'] == 1)&(d['Context'] == 1)
    nogo2_hit_idx = (d['Choice'] == 0) & (d['Category ID'] == 0)&(d['Context'] == 1)

    go2_error_idx = (d['Choice'] == 1) & (d['Category ID'] == 0)&(d['Context'] == 1)
    nogo2_error_idx = (d['Choice'] == 0) & (d['Category ID'] == 1)&(d['Context'] == 1)

    trial_idx = {'T5 : Go / Go : Hit' : go_hit_idx, 'T5 : Go / Nogo : Miss':nogo_error_idx, 'T5 : NoGo / NoGo : CR' : nogo_hit_idx , \
              'T5 : NoGo / Go : FA' : go_error_idx,  'T8 : Go / Go : Hit' : go2_hit_idx, 'T8 : Go / Nogo : Miss':nogo2_error_idx, 'T8 : NoGo / NoGo : CR' : nogo2_hit_idx , \
              'T8 : NoGo / Go : FA' : go2_error_idx}

    
    return trial_idx


def generate_column_label(d) :
    
    trial_idx = produce_trial_index(d)


    # Add corresponding labels to trial in each DataFrame.
    #labels = [k for k in range(1,9)]

    condition = 1
    trial_label = np.zeros(len(d))
    for idx in trial_idx.values() :
        trial_label += idx*condition
        condition += 1

    if 'Label' not in d :
        d.insert(6, 'Label', trial_label)
    # Add a column 'Label' to full DataFrames
    # 1 : Go / Go Hit,  2 : Go / Nogo : Miss,   3 :NoGo / NoGo : CR , 4: NoGo / Go : FA
    # Stimuli category / Animal response
    

In [35]:
files = ['M01','M02','M03','M04','M05','M07_Region1','M07_Region2','M08_Region1','M08_Region2',\
       'M09_Region1','M09_Region2','M10_Region1','M11_Region1','M11_Region2']

df_t8 = [pd.read_pickle('Pickles/' + file + '_T8') for file in files]
df_t5 = [pd.read_pickle('Pickles/' + file + '_T5') for file in files]

data_t8 = {files[i]:df_t8[i] for i in range(len(files))}
data_t5 = {files[i]:df_t5[i] for i in range(len(files))}

In [37]:
for file in files :
    data_t5[file] = remove_NaN(data_t5[file])
    data_t8[file] = remove_NaN(data_t8[file])
    
    data_t5[file]['Category ID'] =  (data_t5[file]['Category ID'] - 1)//4
    data_t8[file]['Category ID'] =  (data_t8[file]['Category ID'] - 1)//4
    
    data_t5[file]['Stim ID'] = (data_t5[file]['Stim ID']-1)//6
    data_t8[file]['Stim ID'] = (data_t8[file]['Stim ID']-1)//6
        
    data_t8[file]['Reward'] = data_t8[file]['Reward'].fillna(0) # Some zero values of the Reward column are badly referenced NaN 
    data_t5[file]['Reward'] = data_t5[file]['Reward'].fillna(0)
    
    
    if 'Context' not in data_t8[file] :
        data_t8[file].insert(8, 'Context', np.ones(len(data_t8[file])))
        
    if 'Context' not in data_t5[file] :
        data_t5[file].insert(8, 'Context', np.zeros(len(data_t5[file])))
        
data_full = {file:pd.concat([data_t5[file],data_t8[file]],ignore_index=True) for file in files} 

/var/folders/fd/x7ht4zdn0yz2xb9997swqj0r0000gn/T/ipykernel_95618/1605717077.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_t8[file]['Reward'] = data_t8[file]['Reward'].fillna(0) # Some zero values of the Reward column are badly referenced NaN
/var/folders/fd/x7ht4zdn0yz2xb9997swqj0r0000gn/T/ipykernel_95618/1605717077.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_t5[file]['Reward'] = data_t5[file]['Reward'].fillna(0)
/var/folders/fd/x7ht4zdn0yz2xb9997swqj0r0000gn/T/ipykernel_95618/1605717077.py:6: SettingWithCopyWarning: 
A value is

In [39]:
# Label each trial with its corresponding condition.
# We have a total of 2*2 + 2*2 = 8 conditions for both tasks (see produce_trial_index function).
for file in files :
    generate_column_label(data_full[file])


In [41]:
with open('Dataframes/df_full.pickle', 'wb') as f:
    pickle.dump(data_full, f)

with open('Dataframes/df_t5.pickle', 'wb') as f:
    pickle.dump(data_t5, f)

with open('Dataframes/df_t8.pickle', 'wb') as f:
    pickle.dump(data_t8, f)